# March Machine Learning Mania
In this notebook, we:
- Try different models/features and assess their performance.
- Simulate multiple 2024 March Madness brackets.

## Imports

In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from gc import collect
import os
import sys
from tqdm import tqdm

from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVR, SVC
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.metrics import mean_squared_error, r2_score, log_loss, accuracy_score, confusion_matrix, classification_report
from xgboost import XGBRegressor, XGBClassifier

# display 100 rows and 100 columns
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 300)

# global random seed
SEED = 0

# set numpy seed
np.random.seed(SEED)

## Load Data

In [11]:
# root dirs
root = 'data/'
mroot = 'data/mens/'
wroot = 'data/womens/'

# load in features compact
features = pd.read_csv('data/processed/features_detailed.csv')

# team names
# teams = pd.concat([pd.read_csv(mroot + 'MTeams.csv'), pd.read_csv(wroot + 'WTeams.csv')], ignore_index=True)

# create a map for team names
# team_map = teams.set_index('TeamID')['TeamName']

## Rearranging the Data
When we input a new row into our model (for the 2024 bracket), the winner could be the first or the second team in the row that we input into our model. Currently, all rows have the winning team on the left, and this will cause our model to learn this locational information. We need to rearrange the order of winners and losers, to where it is essentially random to prevent the model from learning this.

In [12]:
# add Lseed_diff and LLoc
features['Lseed_diff'] = -features['Wseed_diff']
features['LLoc'] = features['WLoc'].map({'H': 'A', 'A': 'H', 'N': 'N'})

In [14]:
# drop columns that can't be used as features
features = features.drop(columns=(['DayNum', 'WRegion', 'LRegion', 'abs_seed_diff', 'Wseed_win_prob', 'Lseed_win_prob']))

In [15]:
# rename W, L to A, B
features.columns = [x.replace('W', 'A_', 1) if x[0] == 'W' else x for x in features.columns]
features.columns = [x.replace('L', 'B_', 1) if x[0] == 'L' else x for x in features.columns]

# check
features.head()

,Season,A_TeamID,A_Score,B_TeamID,B_Score,A_Loc,NumOT,A_Seed,B_Seed,A_RegionSeed,B_RegionSeed,A_PlayIn,B_PlayIn,A_num_games,A_win_ratio,A_avg_Score_for,A_avg_Score_against,A_std_Score_for,A_std_Score_against,A_avg_FGM_for,A_avg_FGM_against,A_std_FGM_for,A_std_FGM_against,A_avg_FGA_for,A_avg_FGA_against,A_std_FGA_for,A_std_FGA_against,A_avg_FGM3_for,A_avg_FGM3_against,A_std_FGM3_for,A_std_FGM3_against,A_avg_FGA3_for,A_avg_FGA3_against,A_std_FGA3_for,A_std_FGA3_against,A_avg_FTM_for,A_avg_FTM_against,A_std_FTM_for,A_std_FTM_against,A_avg_FTA_for,A_avg_FTA_against,A_std_FTA_for,A_std_FTA_against,A_avg_OR_for,A_avg_OR_against,A_std_OR_for,A_std_OR_against,A_avg_DR_for,A_avg_DR_against,A_std_DR_for,A_std_DR_against,A_avg_Ast_for,A_avg_Ast_against,A_std_Ast_for,A_std_Ast_against,A_avg_TO_for,A_avg_TO_against,A_std_TO_for,A_std_TO_against,A_avg_Stl_for,A_avg_Stl_against,A_std_Stl_for,A_std_Stl_against,A_avg_Blk_for,A_avg_Blk_against,A_std_Blk_for,A_std_Blk_against,A_avg_PF_for,A_avg_PF_against,A_std_PF_for,A_std_PF_against,A_avg_diff,A_std_diff,A_home_win_ratio,A_away_win_ratio,A_neutral_win_ratio,A_home_missing,A_away_missing,A_neutral_missing,A_1_score_game_missing,A_ot_game_missing,A_1_score_game_ratio,A_1_score_win_ratio,A_ot_ratio,A_ot_win_ratio,A_recent_win_ratio,A_recent_avg_Score_for,A_recent_avg_Score_against,A_recent_std_Score_for,A_recent_std_Score_against,A_recent_avg_FGM_for,A_recent_avg_FGM_against,A_recent_std_FGM_for,A_recent_std_FGM_against,A_recent_avg_FGA_for,A_recent_avg_FGA_against,A_recent_std_FGA_for,A_recent_std_FGA_against,A_recent_avg_FGM3_for,A_recent_avg_FGM3_against,A_recent_std_FGM3_for,A_recent_std_FGM3_against,A_recent_avg_FGA3_for,A_recent_avg_FGA3_against,A_recent_std_FGA3_for,A_recent_std_FGA3_against,A_recent_avg_FTM_for,A_recent_avg_FTM_against,A_recent_std_FTM_for,A_recent_std_FTM_against,A_recent_avg_FTA_for,A_recent_avg_FTA_against,A_recent_std_FTA_for,A_recent_std_FTA_against,A_recent_avg_OR_for,A_recent_avg_OR_against,A_recent_std_OR_for,A_recent_std_OR_against,A_recent_avg_DR_for,A_recent_avg_DR_against,A_recent_std_DR_for,A_recent_std_DR_against,A_recent_avg_Ast_for,A_recent_avg_Ast_against,A_recent_std_Ast_for,A_recent_std_Ast_against,A_recent_avg_TO_for,A_recent_avg_TO_against,A_recent_std_TO_for,A_recent_std_TO_against,A_recent_avg_Stl_for,A_recent_avg_Stl_against,A_recent_std_Stl_for,A_recent_std_Stl_against,A_recent_avg_Blk_for,A_recent_avg_Blk_against,A_recent_std_Blk_for,A_recent_std_Blk_against,A_recent_avg_PF_for,A_recent_avg_PF_against,A_recent_std_PF_for,A_recent_std_PF_against,A_recent_avg_diff,A_conf_champs,B_num_games,B_win_ratio,B_avg_Score_for,B_avg_Score_against,B_std_Score_for,B_std_Score_against,B_avg_FGM_for,B_avg_FGM_against,B_std_FGM_for,B_std_FGM_against,B_avg_FGA_for,B_avg_FGA_against,B_std_FGA_for,B_std_FGA_against,B_avg_FGM3_for,B_avg_FGM3_against,B_std_FGM3_for,B_std_FGM3_against,B_avg_FGA3_for,B_avg_FGA3_against,B_std_FGA3_for,B_std_FGA3_against,B_avg_FTM_for,B_avg_FTM_against,B_std_FTM_for,B_std_FTM_against,B_avg_FTA_for,B_avg_FTA_against,B_std_FTA_for,B_std_FTA_against,B_avg_OR_for,B_avg_OR_against,B_std_OR_for,B_std_OR_against,B_avg_DR_for,B_avg_DR_against,B_std_DR_for,B_std_DR_against,B_avg_Ast_for,B_avg_Ast_against,B_std_Ast_for,B_std_Ast_against,B_avg_TO_for,B_avg_TO_against,B_std_TO_for,B_std_TO_against,B_avg_Stl_for,B_avg_Stl_against,B_std_Stl_for,B_std_Stl_against,B_avg_Blk_for,B_avg_Blk_against,B_std_Blk_for,B_std_Blk_against,B_avg_PF_for,B_avg_PF_against,B_std_PF_for,B_std_PF_against,B_avg_diff,B_std_diff,B_home_win_ratio,B_away_win_ratio,B_neutral_win_ratio,B_home_missing,B_away_missing,B_neutral_missing,B_1_score_game_missing,B_ot_game_missing,B_1_score_game_ratio,B_1_score_win_ratio,B_ot_ratio,B_ot_win_ratio,B_recent_win_ratio,B_recent_avg_Score_for,B_recent_avg_Score_against,B_recent_std_Score_for,B_recent_std_Score_against,B_recent_avg_FGM_for,B_recent_avg_FGM_against,B_recent_std_FGM_for,B_recent_std_FGM_against,B_recent_avg_FGA_for,B_recent_avg_

In [16]:
# choose (half) random rows to flip
np.random.seed(SEED)
flip = np.random.choice(features.index, int(features.shape[0] / 2), replace=False)

# flip A, B cols
for i, col in enumerate(features.columns):
    if col[0] == 'A':
        features.loc[flip, col], features.loc[flip, col.replace('A_', 'B_', 1)] = features.loc[flip, col.replace('A_', 'B_', 1)].values, features.loc[flip, col].values

# delete vars
del flip, col
collect()

# rearrange cols
features = features.reindex(sorted(features.columns), axis=1)

# check
features.head()

,A_1_score_game_missing,A_1_score_game_ratio,A_1_score_win_ratio,A_Loc,A_PlayIn,A_RegionSeed,A_Score,A_Seed,A_TeamID,A_avg_Ast_against,A_avg_Ast_for,A_avg_Blk_against,A_avg_Blk_for,A_avg_DR_against,A_avg_DR_for,A_avg_FGA3_against,A_avg_FGA3_for,A_avg_FGA_against,A_avg_FGA_for,A_avg_FGM3_against,A_avg_FGM3_for,A_avg_FGM_against,A_avg_FGM_for,A_avg_FTA_against,A_avg_FTA_for,A_avg_FTM_against,A_avg_FTM_for,A_avg_OR_against,A_avg_OR_for,A_avg_PF_against,A_avg_PF_for,A_avg_Score_against,A_avg_Score_for,A_avg_Stl_against,A_avg_Stl_for,A_avg_TO_against,A_avg_TO_for,A_avg_diff,A_away_missing,A_away_win_ratio,A_conf_champs,A_home_missing,A_home_win_ratio,A_neutral_missing,A_neutral_win_ratio,A_num_games,A_ot_game_missing,A_ot_ratio,A_ot_win_ratio,A_recent_avg_Ast_against,A_recent_avg_Ast_for,A_recent_avg_Blk_against,A_recent_avg_Blk_for,A_recent_avg_DR_against,A_recent_avg_DR_for,A_recent_avg_FGA3_against,A_recent_avg_FGA3_for,A_recent_avg_FGA_against,A_recent_avg_FGA_for,A_recent_avg_FGM3_against,A_recent_avg_FGM3_for,A_recent_avg_FGM_against,A_recent_avg_FGM_for,A_recent_avg_FTA_against,A_recent_avg_FTA_for,A_recent_avg_FTM_against,A_recent_avg_FTM_for,A_recent_avg_OR_against,A_recent_avg_OR_for,A_recent_avg_PF_against,A_recent_avg_PF_for,A_recent_avg_Score_against,A_recent_avg_Score_for,A_recent_avg_Stl_against,A_recent_avg_Stl_for,A_recent_avg_TO_against,A_recent_avg_TO_for,A_recent_avg_diff,A_recent_std_Ast_against,A_recent_std_Ast_for,A_recent_std_Blk_against,A_recent_std_Blk_for,A_recent_std_DR_against,A_recent_std_DR_for,A_recent_std_FGA3_against,A_recent_std_FGA3_for,A_recent_std_FGA_against,A_recent_std_FGA_for,A_recent_std_FGM3_against,A_recent_std_FGM3_for,A_recent_std_FGM_against,A_recent_std_FGM_for,A_recent_std_FTA_against,A_recent_std_FTA_for,A_recent_std_FTM_against,A_recent_std_FTM_for,A_recent_std_OR_against,A_recent_std_OR_for,A_recent_std_PF_against,A_recent_std_PF_for,A_recent_std_Score_against,A_recent_std_Score_for,A_recent_std_Stl_against,A_recent_std_Stl_for,A_recent_std_TO_against,A_recent_std_TO_for,A_recent_win_ratio,A_seed_diff,A_std_Ast_against,A_std_Ast_for,A_std_Blk_against,A_std_Blk_for,A_std_DR_against,A_std_DR_for,A_std_FGA3_against,A_std_FGA3_for,A_std_FGA_against,A_std_FGA_for,A_std_FGM3_against,A_std_FGM3_for,A_std_FGM_against,A_std_FGM_for,A_std_FTA_against,A_std_FTA_for,A_std_FTM_against,A_std_FTM_for,A_std_OR_against,A_std_OR_for,A_std_PF_against,A_std_PF_for,A_std_Score_against,A_std_Score_for,A_std_Stl_against,A_std_Stl_for,A_std_TO_against,A_std_TO_for,A_std_diff,A_win_ratio,B_1_score_game_missing,B_1_score_game_ratio,B_1_score_win_ratio,B_Loc,B_PlayIn,B_RegionSeed,B_Score,B_Seed,B_TeamID,B_avg_Ast_against,B_avg_Ast_for,B_avg_Blk_against,B_avg_Blk_for,B_avg_DR_against,B_avg_DR_for,B_avg_FGA3_against,B_avg_FGA3_for,B_avg_FGA_against,B_avg_FGA_for,B_avg_FGM3_against,B_avg_FGM3_for,B_avg_FGM_against,B_avg_FGM_for,B_avg_FTA_against,B_avg_FTA_for,B_avg_FTM_against,B_avg_FTM_for,B_avg_OR_against,B_avg_OR_for,B_avg_PF_against,B_avg_PF_for,B_avg_Score_against,B_avg_Score_for,B_avg_Stl_against,B_avg_Stl_for,B_avg_TO_against,B_avg_TO_for,B_avg_diff,B_away_missing,B_away_win_ratio,B_conf_champs,B_home_missing,B_home_win_ratio,B_neutral_missing,B_neutral_win_ratio,B_num_games,B_ot_game_missing,B_ot_ratio,B_ot_win_ratio,B_recent_avg_Ast_against,B_recent_avg_Ast_for,B_recent_avg_Blk_against,B_recent_avg_Blk_for,B_recent_avg_DR_against,B_recent_avg_DR_for,B_recent_avg_FGA3_against,B_recent_avg_FGA3_for,B_recent_avg_FGA_against,B_recent_avg_FGA_for,B_recent_avg_FGM3_against,B_recent_avg_FGM3_for,B_recent_avg_FGM_against,B_recent_avg_FGM_for,B_recent_avg_FTA_against,B_recent_avg_FTA_for,B_recent_avg_FTM_against,B_recent_avg_FTM_for,B_recent_avg_OR_against,B_recent_avg_OR_for,B_recent_avg_PF_against,B_recent_avg_PF_for,B_recent_avg_Score_against,B_recent_avg_Score_for,B_recent_avg_Stl_against,B_recent_avg_Stl_for,B_recent_avg_TO_against,B_recent_avg_TO_for,B_recent_avg_diff,B_recent_std_Ast_against,B_recent_s

In [17]:
# one-hot encode A_Loc col, drop loc cols
features = pd.get_dummies(features, columns=['A_Loc'], dtype=int)
features = features.drop(columns=['A_Loc_N', 'B_Loc'])

# one-hot encode 'round' col, drop one col
features = pd.get_dummies(features, columns=['round'], dtype=int)
features = features.drop(columns=['round_1'])

## Create Labels

In [18]:
# create regression label, drop cols
features['A_score_diff'] = features['A_Score'] - features['B_Score']
features = features.drop(columns=['A_Score', 'B_Score'])

# create binary label
features['A_win'] = features['A_score_diff'].apply(lambda x: 1 if x > 0 else 0)

# check
features.head()[['A_score_diff', 'A_win']]

,A_score_diff,A_win
0,29,1
1,-13,0
2,-6,0
3,2,1
4,-5,0


In [21]:
# create adjusted score diff col (score diff is inversely scaled by NumOT periods), drop NumOT
features['A_score_diff_adj'] = features['A_score_diff'] / (2 ** features['NumOT'])

# check
features[features['NumOT'] > 0][['A_score_diff', 'NumOT', 'A_score_diff_adj']].sort_values(by='NumOT', ascending=False).head()

,A_score_diff,NumOT,A_score_diff_adj
819,2,2,0.50
184,-6,2,-1.50
864,4,2,1.00
542,-7,2,-1.75
490,5,2,1.25


__score_diff__ and __win__ can both be used as labels. They are both calculated with respect to team A.

In [22]:
# drop cols now that we have adjusted score diff
features = features.drop(columns=['A_score_diff', 'NumOT'])

## Differenced Features
In combination with the team A/B data, we will also try training models using the differences between both teams' aggregated stats.

In [10]:
# # create diff cols (features between 2 teams)
# cols_to_diff = ['1_pos_game_ratio', '1_pos_win_ratio', 'away_win_ratio', 'home_win_ratio', 'max_loss_diff', 'max_win_diff', 'mean_diff', 'mean_papg', 'mean_ppg', 
#                 'neutral_win_ratio', 'num_games', 'ot_ratio', 'ot_win_ratio', 'recent_mean_pts_against', 'recent_mean_pts_for', 'recent_mean_score_diff', 'recent_std_pts_against', 
#                 'recent_std_pts_for', 'recent_std_score_diff', 'recent_win_ratio', 'std_diff', 'std_papg', 'std_ppg', 'win_ratio', 'seed_win_prob']

# # features to be kept but shouldn't be differenced
# cols_to_keep = ['A_TeamID', 'A_FullSeed', 'A_1_pos_loss_missing', 'A_1_pos_win_missing', 'A_ot_loss_missing', 'A_ot_win_missing', 'A_PlayIn', 'B_TeamID', 'B_FullSeed', 
#                 'B_1_pos_loss_missing', 'B_1_pos_win_missing', 'B_ot_loss_missing', 'B_ot_win_missing', 'B_PlayIn', 'A_Loc_A', 'A_Loc_H', 'round_2', 'round_3', 'round_4', 'round_5', 
#                 'round_6', 'score_diff_adj', 'win']

# # create diff df
# features_diff = features[cols_to_keep].copy()

# # create diff cols
# for col in cols_to_diff:
#     features_diff[col + '_diff'] = features['A_' + col] - features['B_' + col]

# # check
# features_diff.head()

## Split Data

In [33]:
# split on gender
mfeatures = features[features['A_TeamID'] < 3000]
wfeatures = features[features['A_TeamID'] >= 3000]
# del features

# same for diff data
# mfeatures_diff = features_diff[features_diff['A_TeamID'] < 3000]
# wfeatures_diff = features_diff[features_diff['A_TeamID'] >= 3000]

## Chalk Bracket
Here, we will simply predict the better seed. If seeds are equal (in rounds 5 and 6), we will predict the team with the better win_ratio.

From the 38 NCAA Men's tournaments that we have access to, the better seed wins __70.62%__ of the time. In the women's, the better seed wins __76.51%__ of the time. In conjunction to predicting the better seed (when seeds are different), we will predict a full __chalk__ bracket by taking the team with a higher regular season win ratio when the seeds are the same.

In [24]:
def get_dummy_preds(data):
    """
    Function to get dummy predictions based on seed and win percentage
    """

    # create a container
    dummy_preds = []

    # loop through the dataframe based on conditions
    for idx, row in data.iterrows():
        if data.loc[idx, "A_Seed"] < data.loc[idx, "B_Seed"]:
            dummy_preds.append(1)
        elif data.loc[idx, "A_Seed"] > data.loc[idx, "B_Seed"]:
            dummy_preds.append(0)
        else:
            if data.loc[idx, "A_win_ratio"] > data.loc[idx, "B_win_ratio"]:
                dummy_preds.append(1)
            else:
                dummy_preds.append(0)
    
    return np.array(dummy_preds)

### Men's

In [26]:
# get dummy preds
mchalk_preds = get_dummy_preds(mfeatures)

# compare preds to win col
mchalk_acc = accuracy_score(mfeatures['A_win'], mchalk_preds)

print(f"Accuracy of dummy predictions: {mchalk_acc*100:.2f}%.")

# delete vars
del mchalk_preds, mchalk_acc

Accuracy of dummy predictions: 70.61%.


### Women's

In [27]:
# get dummy preds
wchalk_preds = get_dummy_preds(wfeatures)

# compare preds to win col
wchalk_acc = accuracy_score(wfeatures['A_win'], wchalk_preds)

print(f"Accuracy of dummy predictions: {wchalk_acc*100:.2f}%")

# delete vars
del wchalk_preds, wchalk_acc

Accuracy of dummy predictions: 78.02%


In [35]:
# drop cols that may not be useful features
mfeatures = mfeatures.drop(columns=(['Season', 'A_seed_diff', 'B_seed_diff', 'A_RegionSeed', 'B_RegionSeed']))
wfeatures = wfeatures.drop(columns=(['Season', 'A_seed_diff', 'B_seed_diff', 'A_RegionSeed', 'B_RegionSeed']))

## Regression/Classification
This problem can be treated as a regression problem (label is __score differential__). It can also be treated as binary classification (label is __win/loss__).

We will try the original aggregated stats for both team A and B as features (mfeatures/wfeatures), and also the differences between the two teams' stats (mcomp_diff/wcomp_diff).

In [36]:
# non feature cols
non_feat_cols = ['A_TeamID', 'B_TeamID', 'A_win', 'A_score_diff_adj']

# look at num feats for the 2 datasets
print(f'Features of both team A and B: {mfeatures.shape[1] - len(non_feat_cols)}')
# print(f'Features of the DIFFERENCE between team A and B: {mfeatures_diff.shape[1] - len(non_feat_cols)}')

Features of both team A and B: 273


### Data Preprocessing

In [39]:
# create feature order list so our models are trained on the same feature order
sorted_cols = sorted(mfeatures.columns)
sorted_cols.remove('A_Loc_A')
sorted_cols.remove('A_Loc_H')
sorted_cols.remove('A_score_diff_adj')
sorted_cols.remove('A_win')
sorted_cols.extend(['A_Loc_A', 'A_Loc_H', 'A_score_diff_adj', 'A_win'])

# same for diff
# diff_cols = sorted(mfeatures_diff.columns)
# diff_cols.remove('A_Loc_A')
# diff_cols.remove('A_Loc_H')
# diff_cols.remove('score_diff_adj')
# diff_cols.remove('win')
# diff_cols.extend(['A_Loc_A', 'A_Loc_H', 'score_diff_adj', 'win'])

In [40]:
# function that takes in estimator, data, models_df, tournament, test_size
def run_model(estimator, features, regression, models_df, tournament, folds=5):
    """
    Run a model on data and save results to models_df.

    Parameters
    ----------
    estimator : sklearn estimator
        Estimator to use for modeling.
    data : pd.DataFrame
        Data to model.
    regression : bool
        Whether to model score differential (regression) or win/loss (classification).
    models_df : pd.DataFrame
        DataFrame to save results to.
    tournament : str
        Gender - 'M' or 'F'.
    folds : int
        Number of cross-validation folds to use.

    Returns
    -------
    models_df.tail() : pd.DataFrame
        Last 5 rows of dataframe with results from model.
    """
    
    # create copy to avoid modification
    data = features.copy()

    # define which col order to use
    if data.shape[1] == 277:
        data = data[sorted_cols]
    # elif data.shape[1] == 48:
    #     data = data[diff_cols]
    # else:
    #     print(f'Number of features: {data.shape[1]}')
    #     raise ValueError('Invalid number of features.')

    # drop unused cols
    data = data.drop(columns=['A_TeamID', 'B_TeamID'])

    # define cross-validation
    kf = KFold(n_splits=folds, shuffle=True, random_state=SEED)

    # create scaler
    scaler = MinMaxScaler()

    # predicting score differential
    if regression:
        # define X and y
        X = data.drop(columns=['A_score_diff_adj', 'A_win'])
        y = data['A_score_diff_adj']

        # initialize lists to store metrics
        rmse_scores_train = []
        rmse_scores_test = []
        r2_scores_train = []
        r2_scores_test = []
        acc_scores_train = []
        acc_scores_test = []

        for train_index, test_index in kf.split(X):
            # split data
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            # scale data
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)

            # fit model
            estimator.fit(X_train_scaled, y_train)

            # predict
            train_preds = estimator.predict(X_train_scaled)
            test_preds = estimator.predict(X_test_scaled)

            # evaluate
            rmse_scores_train.append(mean_squared_error(y_train, train_preds, squared=False))
            rmse_scores_test.append(mean_squared_error(y_test, test_preds, squared=False))
            r2_scores_train.append(r2_score(y_train, train_preds))
            r2_scores_test.append(r2_score(y_test, test_preds))
            acc_scores_train.append(accuracy_score(np.sign(y_train), np.sign(train_preds)))
            acc_scores_test.append(accuracy_score(np.sign(y_test), np.sign(test_preds)))

        # average the metrics across folds
        train_rmse = np.mean(rmse_scores_train)
        test_rmse = np.mean(rmse_scores_test)
        train_r2 = np.mean(r2_scores_train)
        test_r2 = np.mean(r2_scores_test)
        train_acc = np.mean(acc_scores_train)
        test_acc = np.mean(acc_scores_test)
        
        # Save results to models_df
        models_df.loc[len(models_df.index)] = [tournament, 'A_score_diff_adj', estimator, X.shape[1], X.columns.to_list(), folds, train_r2, test_r2, train_rmse, test_rmse, train_acc, test_acc]

        # delete variables
        collect()
        
        return
    
    # predicting win/loss
    else:
        # define X and y
        X = data.drop(columns=['A_score_diff_adj', 'A_win'])
        y = data['A_win']

        # initialize lists to store metrics
        log_loss_train = []
        log_loss_test = []
        acc_scores_train = []
        acc_scores_test = []

        for train_index, test_index in kf.split(X):
            # split data
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            # scale data
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)

            # fit model
            estimator.fit(X_train_scaled, y_train)

            # predict
            train_preds = estimator.predict(X_train_scaled)
            test_preds = estimator.predict(X_test_scaled)


            # evaluate
            log_loss_train.append(log_loss(y_train, train_preds))
            log_loss_test.append(log_loss(y_test, test_preds))
            acc_scores_train.append(accuracy_score(np.sign(y_train), np.sign(train_preds)))
            acc_scores_test.append(accuracy_score(np.sign(y_test), np.sign(test_preds)))

        # average the metrics across folds
        train_log_loss = np.mean(log_loss_train)
        test_log_loss = np.mean(log_loss_test)
        train_acc = np.mean(acc_scores_train)
        test_acc = np.mean(acc_scores_test)
        
        # save results to models_df
        models_df.loc[len(models_df.index)] = [tournament, 'A_win', estimator, X.shape[1], X.columns.to_list(), folds, train_log_loss, test_log_loss, train_acc, test_acc]

        # delete variables
        collect()
        
        return

### Regression (Predicting <ins>Score Differential</ins>)

#### Men's

In [41]:
# create a df to hold regression models
# reg_df = pd.DataFrame(columns=['Tournament', 'Label', 'Model', 'Num_Features', 'Features', 'Num_CV_Folds', 'Train_R2', 'Val_R2', 'Train_RMSE', 'Val_RMSE', 'Train_Acc', 'Val_Acc'])

# load reg df
reg_df = pd.read_csv('models/regression_models_detailed.csv')

In [45]:
# define regression models
models = [LinearRegression(n_jobs=-1), RandomForestRegressor(n_jobs=-1), XGBRegressor(n_jobs=-1), SVR(), KNeighborsRegressor(n_jobs=-1)]
datasets = [mfeatures] # , mfeatures_diff

# run regression models
for model in tqdm(models, desc='Model', file=sys.stdout):
    for dataset in datasets:
        # run model
        run_model(estimator=model, features=dataset, regression=True, models_df=reg_df, tournament='M')

Model: 100%|██████████| 5/5 [00:09<00:00,  1.96s/it]


In [46]:
# inspect
reg_df[reg_df['Tournament'] == 'M'].sort_values(by='Val_Acc', ascending=False).head()

,Tournament,Label,Model,Num_Features,Features,Num_CV_Folds,Train_R2,Val_R2,Train_RMSE,Val_RMSE,Train_Acc,Val_Acc
2,M,A_score_diff_adj,"(DecisionTreeRegressor(max_features='auto', ra...",273,"[A_1_score_game_missing, A_1_score_game_ratio,...",5,0.909098,0.176092,4.339546,11.847898,0.941421,0.662379
0,M,A_score_diff_adj,LinearRegression(n_jobs=-1),273,"[A_1_score_game_missing, A_1_score_game_ratio,...",5,0.542976,-0.057931,9.730279,13.366071,0.760323,0.644903
1,M,A_score_diff_adj,LinearRegression(n_jobs=-1),273,"[A_1_score_game_missing, A_1_score_game_ratio,...",5,0.542976,-0.057931,9.730279,13.366071,0.760323,0.644903
3,M,A_score_diff_adj,"XGBRegressor(base_score=0.5, booster='gbtree',...",273,"[A_1_score_game_missing, A_1_score_game_ratio,...",5,0.999990,0.053361,0.044187,12.746555,1.000000,0.640944
4,M,A_score_diff_adj,SVR(),273,"[A_1_score_game_missing, A_1_score_game_ratio,...",5,0.296842,-0.011214,12.069977,13.102798,0.745233,0.624995


#### Women's

In [47]:
# define regression models
models = [LinearRegression(n_jobs=-1), RandomForestRegressor(n_jobs=-1), XGBRegressor(n_jobs=-1), SVR(), KNeighborsRegressor(n_jobs=-1)]
datasets = [wfeatures] # , wfeatures_diff

# run regression models
for model in tqdm(models, desc='Model', file=sys.stdout):
    for dataset in datasets:
        # run model
        run_model(estimator=model, features=dataset, regression=True, models_df=reg_df, tournament='W')

Model: 100%|██████████| 5/5 [00:05<00:00,  1.07s/it]


In [48]:
# inspect
reg_df[reg_df['Tournament'] == 'W'].sort_values(by='Val_Acc', ascending=False).head()

,Tournament,Label,Model,Num_Features,Features,Num_CV_Folds,Train_R2,Val_R2,Train_RMSE,Val_RMSE,Train_Acc,Val_Acc
9,W,A_score_diff_adj,SVR(),273,"[A_1_score_game_missing, A_1_score_game_ratio,...",5,0.385525,0.343453,16.816486,17.321353,0.820207,0.775340
7,W,A_score_diff_adj,"(DecisionTreeRegressor(max_features='auto', ra...",273,"[A_1_score_game_missing, A_1_score_game_ratio,...",5,0.948288,0.621227,4.878229,13.103390,0.950854,0.771682
6,W,A_score_diff_adj,LinearRegression(n_jobs=-1),273,"[A_1_score_game_missing, A_1_score_game_ratio,...",5,0.829278,0.499071,8.862599,15.058234,0.859586,0.765554
8,W,A_score_diff_adj,"XGBRegressor(base_score=0.5, booster='gbtree',...",273,"[A_1_score_game_missing, A_1_score_game_ratio,...",5,1.000000,0.586603,0.005368,13.710980,1.000000,0.758245
10,W,A_score_diff_adj,KNeighborsRegressor(n_jobs=-1),273,"[A_1_score_game_missing, A_1_score_game_ratio,...",5,0.650335,0.502921,12.684380,15.024624,0.783885,0.732657


In [49]:
# save reg_df
reg_df.to_csv('models/regression_models_detailed.csv', index=False)

### Classification (Predicting <ins>Win/Loss</ins>)

In [50]:
# create a df to hold regression models
# class_df = pd.DataFrame(columns=['Tournament', 'Label', 'Model', 'Num_Features', 'Features', 'Num_CV_Folds', 'Train_LogLoss', 'Val_LogLoss', 'Train_Acc', 'Val_Acc'])

# load reg df
class_df = pd.read_csv('models/classification_models_detailed.csv')

#### Men's

In [51]:
# define regression models
models = [LogisticRegression(n_jobs=-1, random_state=SEED), RandomForestClassifier(n_jobs=-1), XGBClassifier(n_jobs=-1), SVC(), KNeighborsClassifier(n_jobs=-1)]
datasets = [mfeatures] # , mfeatures_diff

# run regression models
for model in tqdm(models, desc='Model', file=sys.stdout):
    for dataset in datasets:
        # run model
        run_model(estimator=model, features=dataset, regression=False, models_df=class_df, tournament='M')

Model: 100%|██████████| 5/5 [00:11<00:00,  2.27s/it]


In [52]:
# inspect
class_df[class_df['Tournament'] == 'M'].sort_values(by='Val_LogLoss').head()

,Tournament,Label,Model,Num_Features,Features,Num_CV_Folds,Train_LogLoss,Val_LogLoss,Train_Acc,Val_Acc
2,M,A_win,"XGBClassifier(base_score=0.5, booster='gbtree'...",273,"[A_1_score_game_missing, A_1_score_game_ratio,...",5,9.992007e-16,12.566074,1.000000,0.636179
3,M,A_win,SVC(),273,"[A_1_score_game_missing, A_1_score_game_ratio,...",5,5.349591e+00,12.951475,0.845115,0.625021
0,M,A_win,"LogisticRegression(n_jobs=-1, random_state=0)",273,"[A_1_score_game_missing, A_1_score_game_ratio,...",5,7.702049e+00,13.032293,0.777005,0.622681
1,M,A_win,"(DecisionTreeClassifier(max_features='auto', r...",273,"[A_1_score_game_missing, A_1_score_game_ratio,...",5,9.992007e-16,13.527677,1.000000,0.608338
4,M,A_win,KNeighborsClassifier(n_jobs=-1),273,"[A_1_score_game_missing, A_1_score_game_ratio,...",5,8.422192e+00,14.734256,0.756156,0.573405


#### Women's

In [53]:
# define regression models
models = [LogisticRegression(n_jobs=-1, random_state=SEED), RandomForestClassifier(n_jobs=-1), XGBClassifier(n_jobs=-1), SVC(), KNeighborsClassifier(n_jobs=-1)]
datasets = [wfeatures] # , wfeatures_diff

# run regression models
for model in tqdm(models, desc='Model', file=sys.stdout):
    for dataset in datasets:
        # run model
        run_model(estimator=model, features=dataset, regression=False, models_df=class_df, tournament='W')

Model: 100%|██████████| 5/5 [00:08<00:00,  1.74s/it]


In [54]:
# inspect
class_df[class_df['Tournament'] == 'W'].sort_values(by='Val_LogLoss').head()

,Tournament,Label,Model,Num_Features,Features,Num_CV_Folds,Train_LogLoss,Val_LogLoss,Train_Acc,Val_Acc
5,W,A_win,"LogisticRegression(n_jobs=-1, random_state=0)",273,"[A_1_score_game_missing, A_1_score_game_ratio,...",5,3.763891e+00,7.884883,0.891025,0.771712
8,W,A_win,SVC(),273,"[A_1_score_game_missing, A_1_score_game_ratio,...",5,2.730643e+00,8.012283,0.920941,0.768023
7,W,A_win,"XGBClassifier(base_score=0.5, booster='gbtree'...",273,"[A_1_score_game_missing, A_1_score_game_ratio,...",5,9.992007e-16,8.139686,1.000000,0.764335
6,W,A_win,"(DecisionTreeClassifier(max_features='auto', r...",273,"[A_1_score_game_missing, A_1_score_game_ratio,...",5,9.992007e-16,8.350799,1.000000,0.758222
9,W,A_win,KNeighborsClassifier(n_jobs=-1),273,"[A_1_score_game_missing, A_1_score_game_ratio,...",5,6.821378e+00,10.119883,0.802503,0.707003


In [55]:
# save class_df
class_df.to_csv('models/classification_models_detailed.csv', index=False)

- After manual model inspection, Linear Regression performed the best when we replaced the seed with seed_win_prob and used the team A/B data.

## Simultate N Brackets

In [19]:
# root dirs
root = 'data/'
mroot = 'data/mens/'
wroot = 'data/womens/'

# load data, get slots for 2024, drop play-ins
slots = pd.read_csv(mroot + 'MNCAATourneySlots.csv')
slots = slots[slots['Season'] == 2024]
slots = slots[slots['Slot'].str.contains('R')].reset_index(drop=True)

# load seed data
seeds_2024 = pd.read_csv(root + '2024_tourney_seeds.csv')

# load in 2024 data
df_2024 = pd.read_csv(root + 'processed/2024_features.csv').drop(columns=['Region', 'Season'])

# drop teams not in 2024 tourney
df_2024 = df_2024[df_2024['TeamID'].isin(seeds_2024['TeamID'])]

# drop play in char
df_2024['FullSeed'] = df_2024['FullSeed'].apply(lambda x: x[:3])

# split into mens and womens
mdf_2024 = df_2024[df_2024['TeamID'] < 3000].reset_index(drop=True)
wdf_2024 = df_2024[df_2024['TeamID'] >= 3000].reset_index(drop=True)

# delete vars
del root, mroot, wroot, seeds_2024, df_2024

In [20]:
# men's historical seed differential winning percentages
mens_seed_win = [(-15, 0.013157894736842105),(-14, 0.04276315789473684),(-13, 0.07236842105263158),(-12, 0.10013769889840882),(-11, 0.12790697674418605),(-10, 0.16451214758997013),(-9, 0.2011173184357542),(-8, 0.25),(-7, 0.30364372469635625),(-6, 0.3222764078027236), 
                 (-5, 0.3409090909090909),(-4, 0.3565744192715802),(-3, 0.3722397476340694),(-2, 0.41212997482713576),(-1, 0.45202020202020204),(0, 0.5),(1, 0.547979797979798),(2, 0.5878700251728642),(3, 0.6277602523659306),(4, 0.6434255807284198),(5, 0.6590909090909092), 
                 (6, 0.6777235921972764),(7, 0.6963562753036437),(8, 0.75),(9, 0.7988826815642458),(10, 0.8354878524100299),(11, 0.872093023255814),(12, 0.8998623011015912),(13, 0.9276315789473684),(14, 0.9572368421052632),(15, 0.9868421052631579)]

# women's historical seed differential winning percentages
womens_seed_win = [(-15, 0.01),(-14, 0.017182890855457225),(-13, 0.024365781710914452),(-12, 0.03154867256637168),(-11, 0.0387315634218289),(-10, 0.04591445427728613),(-9, 0.05309734513274336),(-8, 0.17006802721088435),(-7, 0.17197452229299362),(-6, 0.22679185884764624), 
                   (-5, 0.28160919540229884),(-4, 0.30250672536072387),(-3, 0.32340425531914896),(-2, 0.3844744048872972),(-1, 0.44554455445544555),(0, 0.5),(1, 0.5544554455445545),(2, 0.6155255951127028),(3, 0.676595744680851),(4, 0.6974932746392761),(5, 0.7183908045977012), 
                   (6, 0.7732081411523537),(7, 0.8280254777070064),(8, 0.8299319727891157),(9, 0.9469026548672567),(10, 0.9540855457227139),(11, 0.9612684365781711),(12, 0.9684513274336283),(13, 0.9756342182890856),(14, 0.9828171091445428),(15, 0.99)]

In [33]:
def generate_bracket(data, estimator, tournament, num_brackets, slots_df=slots):
    """
    Generate a single bracket for the 2024 NCAA tournament.

    Parameters
    ----------
    data : pd.DataFrame
        Regular season data for the 2024 teams competing in the tournament.
    estimator : sklearn estimator
        Pre-trained estimator to use for modeling.
    tournament : str
        'M' or 'W'.
    num_brackets : int
        Number of brackets to generate.
    slots : pd.DataFrame
        Slots for the 2024 tournament.

    Returns
    -------
    all_brackets : pd.DataFrame
        DataFrame with the predicted outcomes of the tournament.
    
    """

    # get a copies of data to avoid modifying the original
    features = data.copy()

    # define scaler
    scaler = MinMaxScaler()

    # create empty df for all brackets
    all_brackets = pd.DataFrame()

    # loop for each bracket
    for n in range(1, num_brackets+1):
        # create bracket-specific slots table
        slots = slots_df.copy()

        # create empty results for round
        result_df = pd.DataFrame(columns=["Slot", "Team"])

        # 6 rounds in a single bracket
        for i in range(1, 7):
            # get slots for round
            slots_round = slots[slots['Slot'].str.contains(f'R{i}')].reset_index(drop=True)

            # holds data for each matchup
            round_matchups = []

            # loop through the slots
            for idx, row in slots_round.iterrows():
                # get team A and team B
                A = features[features['FullSeed'] == row['StrongSeed']].reset_index(drop=True)
                B = features[features['FullSeed'] == row['WeakSeed']].reset_index(drop=True)

                # rename cols
                A = A.add_prefix('A_')
                B = B.add_prefix('B_')

                # create matchup dataframe
                combined = pd.concat([A, B], axis=1)

                # append combined row to the list
                round_matchups.append(combined)

            # concatenate all matchup rows into a single DataFrame
            round_df = pd.concat(round_matchups, axis=0).reset_index(drop=True)
            
            # calculate seed diff
            round_df['A_seed_diff'] = round_df['B_Seed'] - round_df['A_Seed']

            # get seed win probability
            def get_seed_win_prob(x, tournament=tournament): 
                seed_list = mens_seed_win if tournament == 'M' else womens_seed_win

                seed_win_list = [y[1] for y in seed_list if y[0] == x]
                if seed_win_list:
                    return seed_win_list[0]
                else:
                    print(f"No seed win probability found for seed difference {x}.")
                    return None

            # add cols
            round_df['A_seed_win_prob'] = round_df['A_seed_diff'].apply(get_seed_win_prob)
            round_df['B_seed_win_prob'] = 1 - round_df['A_seed_win_prob']

            # add some cols that were in training data
            round_df['A_Loc_A'] = 0
            round_df['A_Loc_H'] = 0
            round_df[['round_2', 'round_3', 'round_4', 'round_5', 'round_6']] = 0
            if i == 1:
                pass
            else:
                round_df[f'round_{i}'] = 1

            # drop cols
            round_df = round_df.drop(columns=['A_Seed', 'B_Seed', 'A_seed_diff'])

            # add placeholder cols for sorting
            round_df['score_diff_adj'] = 0
            round_df['win'] = 0
            round_df = round_df[sorted_cols]

            # define X and reorder cols
            X = round_df.drop(columns=['A_TeamID', 'A_FullSeed', 'B_TeamID', 'B_FullSeed', 'score_diff_adj', 'win'])

            # scale data
            X = scaler.fit_transform(X)
            
            # predict the outcomes of the round
            preds = estimator.predict_proba(X)

            # if n == 1:
            #     print(f'\n\nround {i}:')
            #     print(f'preds before: {preds}')

            # generate random values, update preds
            random_values = np.random.rand(len(preds))
            preds = (random_values > preds[:, 0]).astype(int)

            # if n == 1:
            #     print(f'random values: {random_values}')
            #     print(f'preds after: {preds}')

            # replace preds with full seed of winning team
            preds = np.where(preds > 0, round_df['A_FullSeed'], round_df['B_FullSeed'])

            for slot, winner_seed in zip(slots_round['Slot'], preds):
                # save results to result_df
                result_df.loc[len(result_df.index)] = [slot, winner_seed]

            # edit slots df for next round
            if i != 6:
                next_round_slots = slots[slots['Slot'].str.contains(f'R{i+1}')]

                for idx, row in next_round_slots.iterrows():
                    # get the teams playing in that slot for the next round
                    team1 = result_df[result_df['Slot'] == row['StrongSeed']]['Team'].values[0]
                    team2 = result_df[result_df['Slot'] == row['WeakSeed']]['Team'].values[0]

                    # update the slots df
                    slots.loc[slots['Slot'] == row['Slot'], 'StrongSeed'] = team1
                    slots.loc[slots['Slot'] == row['Slot'], 'WeakSeed'] = team2

            # drop teams that have been eliminated
            # features = features[features['FullSeed'].isin(result_df['Team'])].reset_index(drop=True)

        # add bracket col
        result_df['Bracket'] = n

        # append to all_brackets
        all_brackets = pd.concat([all_brackets, result_df], axis=0)

    # add tournament col
    all_brackets['Tournament'] = tournament

    return all_brackets


### Men's

In [22]:
# define scaler
scaler = MinMaxScaler()

In [23]:
# train best men's model on entire dataset
mmodel_reg = LinearRegression(n_jobs=-1)

# sort features
mfeatures = mfeatures[sorted_cols]

# define X and y
X = mfeatures.drop(columns=['A_TeamID', 'A_FullSeed', 'B_TeamID', 'B_FullSeed', 'score_diff_adj', 'win'])
y = mfeatures['score_diff_adj']

# scale data
X_scaled = scaler.fit_transform(X)

# fit model
mmodel_reg.fit(X_scaled, y)

# get train metrics
train_preds = mmodel_reg.predict(X_scaled)
r2 = r2_score(y, train_preds)
rmse = mean_squared_error(y, train_preds, squared=False)
acc = accuracy_score(np.sign(y), np.sign(train_preds))

print(f'Model R2: {r2:.4f}')
print(f'Model RMSE: {rmse:.4f}')
print(f'Model Accuracy: {acc:.4f}')

# delete vars
del X, y, X_scaled, train_preds, r2, rmse, acc

Model R2: 0.4284
Model RMSE: 11.1122
Model Accuracy: 0.7192


In [24]:
# classification
mmodel_class = LogisticRegression(n_jobs=-1)

# sort features
mfeatures = mfeatures[sorted_cols]

# define X and y
X = mfeatures.drop(columns=['A_TeamID', 'A_FullSeed', 'B_TeamID', 'B_FullSeed', 'score_diff_adj', 'win'])
y = mfeatures['win']

# scale data
X_scaled = scaler.fit_transform(X)

# fit model
mmodel_class.fit(X_scaled, y)

# get train metrics
train_preds = mmodel_class.predict(X_scaled)
loss = log_loss(y, train_preds)
acc = accuracy_score(y, train_preds)

print(f'Model Log Loss: {loss:.4f}')
print(f'Model Accuracy: {acc:.4f}')

# delete vars
del X, y, X_scaled, train_preds, loss, acc

Model Log Loss: 9.5694
Model Accuracy: 0.7229


In [34]:
# generate men's bracket
# result_m_reg = generate_bracket(data=mdf_2024, estimator=mmodel_reg, tournament='M')
m_brackets = generate_bracket(data=mdf_2024, estimator=mmodel_class, tournament='M', num_brackets=1)

# delete vars
# del mmodel_reg, mmodel_class

In [36]:
m_brackets.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62
Slot,R1W1,R1W2,R1W3,R1W4,R1W5,R1W6,R1W7,R1W8,R1X1,R1X2,R1X3,R1X4,R1X5,R1X6,R1X7,R1X8,R1Y1,R1Y2,R1Y3,R1Y4,R1Y5,R1Y6,R1Y7,R1Y8,R1Z1,R1Z2,R1Z3,R1Z4,R1Z5,R1Z6,R1Z7,R1Z8,R2W1,R2W2,R2W3,R2W4,R2X1,R2X2,R2X3,R2X4,R2Y1,R2Y2,R2Y3,R2Y4,R2Z1,R2Z2,R2Z3,R2Z4,R3W1,R3W2,R3X1,R3X2,R3Y1,R3Y2,R3Z1,R3Z2,R4W1,R4X1,R4Y1,R4Z1,R5WX,R5YZ,R6CH
Team,W01,W02,W03,W04,W12,W06,W10,W09,X01,X15,X14,X04,X12,X11,X10,X08,Y01,Y02,Y14,Y04,Y12,Y11,Y10,Y09,Z01,Z02,Z14,Z13,Z12,Z11,Z07,Z09,W01,W02,W06,W04,X01,X10,X11,X04,Y01,Y02,Y11,Y04,Z01,Z02,Z11,Z12,W01,W06,X01,X11,Y04,Y02,Z12,Z02,W01,X01,Y02,Z12,W01,Z12,W01
Bracket,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Tournament,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M


### Women's

In [37]:
# train best men's model on entire dataset
wmodel_reg = LinearRegression(n_jobs=-1)

# define X and y
X = wfeatures.drop(columns=['A_TeamID', 'A_FullSeed', 'B_TeamID', 'B_FullSeed', 'score_diff_adj', 'win'])
y = wfeatures['score_diff_adj']

# scale data
X_scaled = scaler.fit_transform(X)

# fit model
wmodel_reg.fit(X_scaled, y)

# get train metrics
train_preds = wmodel_reg.predict(X_scaled)
r2 = r2_score(y, train_preds)
rmse = mean_squared_error(y, train_preds, squared=False)
acc = accuracy_score(np.sign(y), np.sign(train_preds))

print(f'Model R2: {r2:.4f}')
print(f'Model RMSE: {rmse:.4f}')
print(f'Model Accuracy: {acc:.4f}')

# delete vars
del X, y, X_scaled, train_preds, r2, rmse, acc

Model R2: 0.6467
Model RMSE: 12.3932
Model Accuracy: 0.7956


In [38]:
# classification
wmodel_class = LogisticRegression(random_state=SEED)

# define X and y
X = wfeatures.drop(columns=['A_TeamID', 'A_FullSeed', 'B_TeamID', 'B_FullSeed', 'score_diff_adj', 'win'])
y = wfeatures['win']

# scale data
X_scaled = scaler.fit_transform(X)

# fit model
wmodel_class.fit(X_scaled, y)

# get train metrics
train_preds = wmodel_class.predict(X_scaled)
loss = log_loss(y, train_preds)
acc = accuracy_score(y, train_preds)

print(f'Model Log Loss: {loss:.4f}')
print(f'Model Accuracy: {acc:.4f}')

# delete vars
del X, y, X_scaled, train_preds, loss, acc

Model Log Loss: 6.8640
Model Accuracy: 0.8013


In [39]:
# generate women's bracket
# result_w_reg = generate_bracket(data=wdf_2024, estimator=wmodel_reg, tournament='W')
w_brackets = generate_bracket(data=wdf_2024, estimator=wmodel_class, tournament='W', num_brackets=1)

# delete vars
# del wmodel_reg, wmodel_class

In [40]:
w_brackets.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62
Slot,R1W1,R1W2,R1W3,R1W4,R1W5,R1W6,R1W7,R1W8,R1X1,R1X2,R1X3,R1X4,R1X5,R1X6,R1X7,R1X8,R1Y1,R1Y2,R1Y3,R1Y4,R1Y5,R1Y6,R1Y7,R1Y8,R1Z1,R1Z2,R1Z3,R1Z4,R1Z5,R1Z6,R1Z7,R1Z8,R2W1,R2W2,R2W3,R2W4,R2X1,R2X2,R2X3,R2X4,R2Y1,R2Y2,R2Y3,R2Y4,R2Z1,R2Z2,R2Z3,R2Z4,R3W1,R3W2,R3X1,R3X2,R3Y1,R3Y2,R3Z1,R3Z2,R4W1,R4X1,R4Y1,R4Z1,R5WX,R5YZ,R6CH
Team,W01,W02,W03,W04,W05,W11,W10,W09,X01,X02,X03,X04,X12,X06,X10,X09,Y01,Y02,Y03,Y04,Y12,Y11,Y10,Y09,Z01,Z02,Z03,Z04,Z05,Z06,Z10,Z09,W01,W02,W03,W05,X01,X02,X06,X04,Y01,Y02,Y03,Y04,Z01,Z10,Z03,Z05,W01,W02,X04,X02,Y01,Y02,Z01,Z03,W01,X02,Y01,Z03,W01,Y01,Y01
Bracket,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Tournament,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W,W


## Submission

In [ ]:
# num brackets
n_brackets = 100000

# combine results
submission = pd.concat([result_m_class, result_w_class])
submission = submission.reset_index(drop=True)
submission.index.names = ['RowId']

# reorder
submission = submission[['Tournament', 'Bracket', 'Slot', 'Team']]

In [ ]:
# save
submission.to_csv('submission.csv')
submission.head()